In [35]:
#%%
## Import Library
import copy
import openai
from azure.core.credentials import AzureKeyCredential
from azure.identity import AzureDeveloperCliCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    HnswParameters,
    PrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticSettings,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)
from azure.storage.blob import BlobServiceClient
from langchain.chains import LLMChain
from langchain.llms import AzureOpenAI 
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import CosmosDBChatMessageHistory
import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError
import json

from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
#from langchain.retrievers import AzureCognitiveSearchRetriever
from langdetect import detect
from langchain.prompts import PromptTemplate
import re
# Create chain to answer questions
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationalRetrievalChain

# Import Azure OpenAI
from langchain.llms import AzureOpenAI 
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

import openai
from azure.core.credentials import AzureKeyCredential
from azure.identity import AzureDeveloperCliCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    HnswParameters,
    PrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticSettings,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)

from azure.storage.blob import BlobServiceClient
from pypdf import PdfReader
from langchain.schema import Document
import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
#import textwrap
import logging
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError
import json


In [16]:
index_name = "credit-proposal"
search_service = "gptdemosearch"
search_api_key = "PcAZcXbX2hJsxMYExc2SnkMFO0D94p7Zw3Qzeu5WjYAzSeDMuR5O"
storage_service = "creditproposal"
storage_api_key = "hJ2qb//J1I1KmVeDHBpwEpnwluoJzm+b6puc5h7k+dnDSFQ0oxuh1qBz+qPB/ZT7gZvGufwRbUrN+ASto6JOCw=="
connect_str = f"DefaultEndpointsProtocol=https;AccountName={storage_service};AccountKey={storage_api_key}"

doc_intell_endpoint = "https://doc-intelligence-test.cognitiveservices.azure.com/"
doc_intell_key = "9fac3bb92b3c4ef292c20df9641c7374"

# set up openai environment
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://pwcjay.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "f282a661571f45a0bdfdcd295ac808e7"

os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = search_service
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = search_api_key
os.environ["AZURE_INDEX_NAME"] = index_name

In [18]:
# set up this varible to mock as front end return sector name
section = 'Executive Summary'

In [26]:
# Core LLM call funcition

def cap(match):
    return(match.group().capitalize())

def proposal_proposal_template_text():
    proposal_proposal_template_text = """
        Read the input prompt, RM note (Keyword:rm_note), rm_note (Keyword: rm_note), document (Keyword: document), and example for this section carefully:
        
        You could refer to Previous Generated Paragraph to update the output Paragraph, when previous_paragraph contains nothing, meaning this generation is the first time, please generate as long as you can.
        Previous Generated Paragraph:
        {previous_paragraph}

        Prompt:
        {prompt}
        ======
        RM Note: (Keyword: rm_note)
        {rm_note}
        ======
        Document: (Keyword: document)
        {document}
        ======
        Example: (Keyword: proposal_example)
        {example}
        ======
        
        Then write paragraph(s) based on the above aggregrated context

        Rules you need to follow:
        1. Don't mention the word "RM Note" and "Component", and don't mention you held a meeting with the client! Instead, you shall say "It is mentioned that"
        2. Don't mention the source of your input (i.e. RM Note (Keyword: rm_note), example, document)
        3. Don't justify your answers
        4. Don't provide suggestion or recommendation by yourself
        5. Provide your answer in English
        6. Breake it to multi-paragraphs if one single paragraph consists of more than 100 words
        7. In the same paragraph, don't input line breaks among the sentences
        8. Don't start with your answer by a title. You must start your paragraph immediately
        9. The example (Keyword: proposal_example) above is just for your reference only to improve your theme, you must not directly copy the content in the examples
        10. If possible, you can use point-form, tables to provide your answer
        11. Don't introduce what this section includes

        Guidance when you do not have the information:
        1. When you don't have the specific information or you need further information (Keyword: further_info), you have to write it in the following format: [RM please helps provide the further information of (Keyword: further_info)], where please supplement the information you need here.
        2. You must not create the information by yourself if you don't have relevant information
        3. You cannot say "It's unclear that", please refer to point 1 for the formatting for requesting further information

        Take a deep breath and work on this step by step
        """
    prompt_template_proposal = PromptTemplate(template=proposal_proposal_template_text, input_variables=["previous_paragraph", "prompt", "rm_note", "document", "example"])

    llm_proposal = AzureChatOpenAI(deployment_name="gpt-35-16k", temperature=0,
                            openai_api_version="2023-05-15", openai_api_base="https://pwcjay.openai.azure.com/")
    
    return prompt_template_proposal, llm_proposal

def core_llm_call(previous_paragraph, prompt, rm_note, document, example):
    prompt_template_proposal, llm_proposal = proposal_proposal_template_text()

    """
    A core function to generate the proposal per section

    Parameters:
    -----------
    prompt: str
    Prompt text for instructing the output based on RM prompt

    rm_note: str
    It contains the information from the RM

    example: str
    Input example for GPt to take it as an example

    """
    #.format(content=content, section=section, context=context)
    
    chain = LLMChain(
        llm=llm_proposal,
        prompt=prompt_template_proposal
    )
    drafted_text = chain({"previous_paragraph": previous_paragraph
                    , "prompt": prompt
                    , "rm_note":rm_note
                    , "document":document
                    , "example":example})['text']
    drafted_text2 = drafted_text.replace("Based on the given information, ", "").replace("It is mentioned that ", "")
    
    #All capital letters for first letter in sentences
    formatter = re.compile(r'(?<=[\.\?!]\s)(\w+)')
    drafted_text2 = formatter.sub(cap, drafted_text2)

    #output the result
    return drafted_text2


In [19]:
def load_json(json_path):
    with open(json_path, "r" ,encoding="utf-8") as f:
        data = json.load(f)
    return data

#This funcition is to prepare the rm note in desired format for web, call by app.py
def web_extract_RM(section, rm_note_txt):
    hierarchy_file_name = "hierarchy_v2.json"

    hierarchy_dict_list = load_json(hierarchy_file_name)

    prompt_template_for_extracting_rm_note = """
    Read the RM note (Keyword: rm_note) to answer the input question (Keyword: Question):
    
    You could follow the below extraction example format to output the answer.
    example (Keyword: example):
    {example}

    rm_note (Keyword: rm_note):
    {rm_note}
    ======
    Question (Keyword: Question):
    {question}
    ======
    
    Then answer the question based on the above aggregrate context

    Guidance when you do not have that information:
    1. When you don't have the specific information based on rm_note (Keyword: rm_note), you have to write it in the following format: [Information not available]
    2. You must not create the information by yourself if you don't have relevant information
    3. You cannot say "It's unclear that", please refer to point 1 for the formatting for requesting further information

    Rules you have to follow:
    1. Please provide your answer in English
    2. do not start your answer with "Based on the given information"
    3. The example (Keyword: proposal_example) above is just for your reference only to improve your theme, you must not directly copy the content in the examples
    4. If possible, try to expand the information provided from the RM
    5. Do not create any figures by make-up 

    Take a deep breath and work on this step by step
    """
    rm_prompt_template = PromptTemplate(template=prompt_template_for_extracting_rm_note, input_variables=["example","rm_note", "question"])

    llm_rm_note = AzureChatOpenAI(deployment_name="gpt-35-16k", temperature=0.1,
                            openai_api_version="2023-05-15", openai_api_base="https://pwcjay.openai.azure.com/")

    output_dict_list = []
    for dictionary in hierarchy_dict_list:
        if dictionary["Section"] == section:
            chain = LLMChain(llm=llm_rm_note, prompt=rm_prompt_template)
            dictionary["Value"] = chain({"example": dictionary["Example"],"rm_note":rm_note_txt, "question": dictionary["Question"]})['text']
            dictionary["Value"] = dictionary["Value"].replace("Based on the given information, ", "")
            output_dict_list.append(dictionary)

    # Create Json file 
    # output_json_name = "GOGOVAN_hierarchy_rm_note.json"
    # json.dump(output_dict_list, open(output_json_name, "w"), indent=4)

    return output_dict_list


In [20]:
rm_note_txt = """Client: Gogovan
Industry: Logistics and Delivery Services
Date: 20 Nov 2023

Meeting Summary:	
Today, I evaluated Gogovan's credit proposal for their expansion plans, technology investments, and working capital needs.	

Client Background:	
Gogovan is a leading logistics and delivery service provider, offering on-demand delivery solutions to individuals and businesses. Established in 2013, the company has rapidly expanded its operations and established a strong presence in the market. Gogovan operates a user-friendly mobile application and web platform, connecting customers with a network of professional drivers and delivery partners.	

Financial Performance:	
Gogovan has demonstrated consistent revenue growth over the past few years, driven by increasing customer adoption and expansion into new markets. The company's financial statements reflect a healthy profitability margin, indicating effective cost management and operational efficiency. Cash flow from operations has been positive, providing a stable source of funds to support working capital requirements and ongoing business operations.	

Market Position and Competitive Landscape:	
Gogovan has successfully positioned itself as a market leader in the logistics and delivery industry, leveraging its strong brand recognition and innovative technology platform. The company has built a robust network of drivers and delivery partners, enabling quick and reliable service fulfillment. Gogovan's competitive advantage lies in its ability to offer cost-effective and flexible solutions tailored to meet the needs of various customer segments, including e-commerce, retail, and individual users.	

Growth Strategy and Market Potential:	
Gogovan has outlined a comprehensive growth strategy focused on expanding its geographical presence, diversifying its service offerings, and enhancing customer experience. The company plans to enter new markets, both domestically and internationally, to capture additional customer segments and increase market share. Gogovan aims to invest in technology and infrastructure improvements to streamline operations, optimize delivery routes, and enhance overall efficiency.	

Risk Assessment:	
The logistics industry is subject to various risks, including intense competition, regulatory changes, and economic downturns. Gogovan has implemented risk mitigation measures such as diversification of services and markets, maintaining strong relationships with key partners, and closely monitoring market trends. Operational risks, such as driver availability, vehicle maintenance, and service disruptions, are managed through rigorous driver screening, continuous training programs, and proactive maintenance schedules. Financial risks are mitigated by maintaining a healthy liquidity position, diversifying funding sources, and prudent financial management practices.	

Credit Request and Repayment Plan:	
Gogovan is requesting a credit facility of $10million to support its expansion plans, technology investments, and working capital needs. The proposed repayment plan consists of regular principal and interest payments over a 3 years term, aligning with the company's projected cash flow generation and financial performance.	

Conclusion:	
Gogovan has demonstrated a strong market position, consistent financial performance, and a well-defined growth strategy. With its robust operational capabilities, innovative technology platform, and customer-centric approach, the company is well-positioned to capitalize on the growing demand for logistics and delivery services. The proposed credit facility, in line with the company's financial projections, will support Gogovan's expansion plans and enable it to maintain its competitive edge in the market.	
"""

In [21]:
web_extract_RM(section ,rm_note_txt)

[{'Section': 'Executive Summary',
  'Sub-section': 'EXECUTIVE SUMMARY',
  'Breakdown': '',
  'Value': 'The executive summary is not available in the given information.',
  'Question': 'Please extract the EXECUTIVE SUMMARY',
  'Example': 'The deal is referred by Pamfleet Group (“Pamfleet”) and our relationship with it can be traced back to early 2006 when the OIC was with Hang Seng Bank.  After joining ICBC (Asia), OIC’d tried a couple of time to cooperate with Pamfleet, but was in vain owing to the more aggressive offer given by Pamfleet’s partner banks, like CITIC Ka Wah Bank.  However, due to ICBCA business scale, network plus OIC’s marketing effort and cordial relationship, Pamfleet agreed to provide an industry / office acquisition project for our consideration. In this project, Pamfleet is to cooperate with Angelo Gordon Group (“Angelo Gordon”), a US investment company dedicated to alternative investment to form a fund, which will be used to acquire Ever Gain Plaza located in Kwai

In [32]:
#%%
schema_file_name = "schema_v2.json"

schema_dict_list = load_json(schema_file_name)
hierarchy_rm_note_list = web_extract_RM(section ,rm_note_txt)

def update_schema_json_by_parts(section ,schema_dict_list, hierarchy_dict_list, update_key='RM Note'):
    """
    This function will update the schema_dict_list (deepcopy) JSON by the specified key (e.g. RM Note).
    You will update the values based on hierarchy_dict_list (eitehr stored the RM info or Document info)
    """

    updated_schema_dict = next((item for item in schema_dict_list if item['Section'] == section), None)
    updated_schema_dict_list = []
    updated_schema_dict_list.append(updated_schema_dict)
    # updated_schema_dict_list = copy.deepcopy(schema_dict_list)
    update_dict = {}
    for l in hierarchy_dict_list:
        section_name = l["Section"]
        if section_name not in update_dict:
            update_dict[section_name] = ""
        if len(l['Value']) > 0:
            if l['Breakdown'] == "":
                update_dict[section_name] += "- {}: {}\n".format(l['Sub-section'], l['Value'])
            else:
                update_dict[section_name] += "- {} ({}): {}\n".format(l['Sub-section'], l['Breakdown'], l['Value'])
    for schema_dict in updated_schema_dict_list:
        if schema_dict['Section'] in update_dict.keys():
            if update_key in schema_dict:
                schema_dict[update_key] = update_dict[schema_dict['Section']]
            else:
                raise ValueError("You do not have key value: {} in your schema_dict_list JSON".format(update_key))
    return updated_schema_dict_list

schema_with_rm_note = update_schema_json_by_parts('Executive Summary', schema_dict_list, web_extract_RM('Executive Summary' ,rm_note_txt), update_key='RM Note')
print(schema_with_rm_note)


[{'Section': 'Executive Summary', 'Component': ['EXECUTIVE SUMMARY', 'Overview of the Proposal'], 'Prompt': '', 'Example': 'The deal is referred by Pamfleet Group (“Pamfleet”) and our relationship with it can be traced back to early 2006 when the OIC was with Hang Seng Bank.  After joining ICBC (Asia), OIC’d tried a couple of time to cooperate with Pamfleet, but was in vain owing to the more aggressive offer given by Pamfleet’s partner banks, like CITIC Ka Wah Bank.  However, due to ICBCA business scale, network plus OIC’s marketing effort and cordial relationship, Pamfleet agreed to provide an industry / office acquisition project for our consideration. In this project, Pamfleet is to cooperate with Angelo Gordon Group (“Angelo Gordon”), a US investment company dedicated to alternative investment to form a fund, which will be used to acquire Ever Gain Plaza located in Kwai Chung (the “Property”) or the companies holding the Property.\nIntroduction of the deal brought by Pamfleet Group

In [40]:
# This funcition is to gen the first run of paragraph with schema_with_rm_note, which is the previous step done

def gen_first_web_and_json(schema_with_both_extract):
    # run with GPT by sectors

    # As this is the first run, there should not have previous_paragraph
    previous_paragraph = ""

    New_schema_dict_list_with_GPT = copy.deepcopy(schema_with_both_extract)
    for l in New_schema_dict_list_with_GPT:
        section = l["Section"]
        prompt = l['Prompt']
        rm_note = l['RM Note']
        document = l['Document']
        example = l['Example']
        l["GPT Output"] = core_llm_call(previous_paragraph, prompt, rm_note, document, example)

        # extract [RM please ...] text out of the output and put to ["RM fill"] key-value
        match = re.search(r"\[RM please .*?\]", l["GPT Output"])
        if match:
            l["RM fill"] = match.group(0)
            l["GPT Output"] = l["GPT Output"].replace(match.group(0), "")
        print(section)
        print(l["GPT Output"])
        print("="*20)

    # json.dump(New_schema_dict_list_with_GPT, open("testing_schema.json", "w"), indent=4)

    return New_schema_dict_list_with_GPT

web_json_by_parts = gen_first_web_and_json(schema_with_rm_note)
print(web_json_by_parts)

Executive Summary
Gogovan is requesting a credit facility of $10 million to support its expansion plans, technology investments, and working capital needs. The proposed repayment plan consists of regular principal and interest payments over a 3-year term, aligning with the company's projected cash flow generation and financial performance. This credit facility will enable Gogovan to maintain its competitive edge in the market and capitalize on the growing demand for logistics and delivery services.

The deal is referred by Pamfleet Group and there is a long-standing relationship between them and our organization. The relationship can be traced back to early 2006 when the OIC was with Hang Seng Bank. After joining ICBC (Asia), there were previous attempts to cooperate with Pamfleet, but they were unsuccessful due to more aggressive offers from Pamfleet's partner banks. However, due to the business scale, network, marketing efforts, and cordial relationship between ICBCA and Pamfleet, th

In [41]:
# This code is for the edit of RM input instruction 
# Takes Json like this : web_json_by_parts : [{'Section': 'Executive Summary', 'Component': ['EXECUTIVE SUMMARY', 'Overview of the Proposal'], 'Prompt': '', 'Example': 'The deal is referred by Pamfleet Group (“Pamfleet”) …… holding the Property.\n', 'RM Note': "- EXECUTIVE SUMMARY: The executive summary is not available in the given information.\n- Overview of the Proposal: … delivery services.\n", 'Document': '', 'GPT Output': "Gogovan … Pamfleet Group. [RM please help provide the further information on the credit facility terms and conditions and the details of the industry/office acquisition project]", 'Others': '', 'Component_Text': 'EXECUTIVE SUMMARY, Overview of the Proposal'}]


def edit_web_json(section, prompt, previous_Json):
    for i in previous_Json:
        # Edit 'Project Overview'
        if section == i['Section']:
            rm_note = i['RM Note']
            document = i['Document']
            example = i['Example']
            i["GPT Output"] = core_llm_call(i["GPT Output"], prompt, rm_note, document, example)
            print(i["GPT Output"])
            print("="*20)
            print_sector = i 
    
    return print_sector

re_write_json = edit_web_json('Executive Summary', "Gogovan is requesting a credit facility of $80 million instead of 10 million", web_json_by_parts)
print(re_write_json)

Gogovan is requesting a credit facility of $80 million instead of the initial request of $10 million. This increased credit facility is intended to support Gogovan's expansion plans, technology investments, and working capital needs. The proposed repayment plan will consist of regular principal and interest payments over a 3-year term, aligning with the company's projected cash flow generation and financial performance. With this credit facility, Gogovan aims to maintain its competitive edge in the market and capitalize on the growing demand for logistics and delivery services.

Furthermore, Pamfleet Group has referred this deal and there is a long-standing relationship between Pamfleet and our organization. The relationship can be traced back to early 2006 when the OIC was with Hang Seng Bank. After joining ICBC (Asia), there were previous attempts to cooperate with Pamfleet, but they were unsuccessful due to more aggressive offers from Pamfleet's partner banks. However, due to the bu